In [17]:
import open3d as o3d
import numpy as np
from collections import defaultdict

from open3d.geometry import PointCloud

# Voxel Downsampling
Group points into voxels and use the centroid for each voxel.

In [4]:
pcd = o3d.io.read_point_cloud('registered_bunny.ply')
voxel_size = 0.001  
pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
o3d.visualization.draw_geometries([pcd_down])

# Random Downsampling

In [8]:
pcd = o3d.io.read_point_cloud("registered_bunny.ply") 
N = len(pcd.points)

# Set sampling ratio
ratio = 0.1
M = int(N * ratio)

indices = np.random.choice(N, M, replace=False)
sampled_pcd = pcd.select_by_index(indices)

o3d.visualization.draw_geometries([sampled_pcd], window_name="Random Sampled Point Cloud")

# Uniform Sampling

In [9]:
pcd = o3d.io.read_point_cloud("registered_bunny.ply") 
downsampled_pcd = pcd.uniform_down_sample(every_k_points=5)
o3d.visualization.draw_geometries([downsampled_pcd])

# FPS
Select points as far as possible.
Use an appropriate M to coverage the geometric information.

In [14]:
pcd = o3d.io.read_point_cloud("registered_bunny.ply") 
downsampled = pcd.farthest_point_down_sample(num_samples=1000)
o3d.visualization.draw_geometries([downsampled])

# Grid Minimum/Maximum Sampling
Instead of choosing the centroid like Voxel Sampling, this method chooses the point with the minimum/maximum height in the grid.

In [ ]:
def grid_min_max_sampling(pcd: PointCloud, voxel_size: float = 0.001, mode: str = 'min'):
    """
    Grid Minimum/Maximum Sampling for a point cloud.
    
    Args:
        pcd (PointCloud): 
            Input point cloud.
        voxel_size (float): 
            Size of voxel grid.
        mode (str): 
            'min' for minimum z sampling, 'max' for maximum z sampling.
    
    Returns:
        samled_pcd (PointCloud): 
            Downsampled point cloud.
    """    
    points = np.asarray(pcd.points)
    voxel_dict = defaultdict(list)
    

    voxel_indices = np.floor(points[:, :3] / voxel_size).astype(np.int32)

    for idx, voxel in enumerate(voxel_indices):
        voxel_key = tuple(voxel)
        voxel_dict[voxel_key].append(points[idx])

    # 2. Select min or max point (based on z) from each voxel
    sampled_points = []
    for pts in voxel_dict.values():
        pts = np.array(pts)
        if mode == 'min':
            selected = pts[np.argmin(pts[:, 2])]  # Min z
        else:
            selected = pts[np.argmax(pts[:, 2])]  # Max z
        sampled_points.append(selected)

    # 3. Return new point cloud
    sampled_pcd = o3d.geometry.PointCloud()
    sampled_pcd.points = o3d.utility.Vector3dVector(np.array(sampled_points))
    return sampled_pcd

In [33]:
sampled = grid_min_max_sampling(pcd)
o3d.visualization.draw_geometries([sampled])

[-0.01525    0.187218  -0.0237782]
[[-16 187 -24]
 [-17 187 -21]
 [-18 187 -20]
 ...
 [-25 111 -21]
 [ 26 112 -12]
 [-83 150   4]]
